In [ ]:
pip install -q faster-whisper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 118.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.3 MB/s eta 0:00:00


In [ ]:
from faster_whisper import WhisperModel
from pydub import AudioSegment
import os
import sys
from datetime import datetime

# 하나의 오디오 파일 처리 함수 (.txt 파일에 문장만 저장 + 날짜 추가)
def process_audio_to_txt(audio_path, model, txt_folder, chunk_length_min=10):
    base_name = os.path.splitext(os.path.basename(audio_path))[0]
    txt_output_path = os.path.join(txt_folder, f"{base_name}.txt")
    os.makedirs(txt_folder, exist_ok=True)

    print(f"\n처리 시작: {base_name}")

    # 현재 날짜 구하기
    today_str = datetime.now().strftime("[%Y-%m-%d]")

    # 오디오 불러오기
    audio = AudioSegment.from_file(audio_path)
    chunk_length_ms = chunk_length_min * 60 * 1000
    total_chunks = len(audio) // chunk_length_ms + 1

    current_offset = 0  # 오프셋 (초 단위)

    with open(txt_output_path, "w", encoding="utf-8") as txt_file:
        txt_file.write(today_str + "\n")  # 날짜 한 줄 추가

        for i in range(total_chunks):
            start = i * chunk_length_ms
            end = min(len(audio), (i + 1) * chunk_length_ms)
            chunk = audio[start:end]

            temp_chunk_path = "temp_chunk.wav"
            chunk.export(temp_chunk_path, format="wav")

            segments, _ = model.transcribe(temp_chunk_path, beam_size=8, language="ko")

            for segment in segments:
                txt_file.write(segment.text.strip() + "\n")
                sys.stdout.write(f"\r문장 저장 중... {segment.text[:30]}...")
                sys.stdout.flush()

            current_offset += (end - start) / 1000.0
            os.remove(temp_chunk_path)

    print(f"\n변환 완료: '{base_name}.txt' 저장됨.")

# ========================
# 메인 실행 코드
# ========================
if __name__ == "__main__":
    model_size = "large-v3"
    model = WhisperModel(model_size, device="cuda", compute_type="float16")

    audio_folder = "."       # 현재 폴더에서 mp3 파일 찾기
    txt_folder = "TXT"       # 결과 txt 저장 폴더

    os.makedirs(txt_folder, exist_ok=True)

    for file in os.listdir(audio_folder):
        if file.endswith(".mp3"):
            audio_file = os.path.join(audio_folder, file)
            process_audio_to_txt(audio_file, model, txt_folder)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

vocabulary.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]


처리 시작: GMT20250725-110032_Recording
문장 저장 중...  바이...
변환 완료: 'GMT20250725-110032_Recording.txt' 저장됨.
